In [133]:
import data_loader as dl

In [134]:
sessions = dl.get_merged_data()

columns_to_drop = ['user_id', 'product_id', 'purchase_id', 'street']
sessions = sessions.drop(columns_to_drop, axis=1)

sessions.to_csv("merged_dataset", sep=' ')